In [1]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib
import numpy as np

In [13]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [14]:
path = r"New_Testing_Protocol_Generalized.csv"
chem_path = r"Chemical Database.csv"

In [15]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [16]:
# the goal is to keep the concentration df the same format as all others, hence if you want to do other method you can 
# just need to slightly modify functions. 

In [17]:
# ideally build a function that will pull a component name from a column this would lead to not needing anything provided in the plan if importing!
# but potential issue when calulating volumes say if you need a stock density (well no cause thats shoudl be stock order specific)
# so everything should be order specfic if inputting from plan and using order from there does that make snes

In [18]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan, unity_filter=True) # for now lets work wit this caase and go back and edit
amounts_added = CreateSamples_renewed.determine_component_amounts(concentration_df, plan)
complete_amounts_added = CreateSamples_renewed.calculate_component_amount_missing(amounts_added, plan)
complete_amounts_added_zeroed = CreateSamples_renewed.nan_amounts(complete_amounts_added)
complete_amounts_added_zeroed

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated component volumes from a component mass using dppc density of nan g/mL
You calculated component volumes from a component mass using dspepeg2000 density of nan g/mL
You calculated component volumes from a component mass using pfh density of 1.691 g/mL
You calculated component volumes from a component mass using ethanol density of 0.7893 g/mL
You calculated component volumes from a component mass using water density of 1.0 g/mL


dppc concentration wtf  dspepeg2000 concentration wtf  \
0                  0.000075                       0.000035   
1                  0.000075                       0.000035   
2                  0.000075                       0.000035   
3                  0.000075                       0.000035   
4                  0.000075                       0.000035   
..                      ...                            ...   
495                0.000075                       0.000035   
496                0.000075                       0.000035   
497                0.000075                       0.000035   
498                0.000075                       0.000035   
499                0.000075                       0.000035   

     pfh concentration wtf  ethanol concentration wtf  \
0                     0.00                   0.000000   
1                     0.00                   0.036842   
2                     0.00                   0.073684   
3                     0.00                   0.110526   
4                     0.00                   0.147368   
..                     ...                        ...   
495                   0.01                   0.552632   
496                   0.01                   0.589474   
497                   0.01                   0.626316   
498                   0.01                   0.663158   
499                   0.01                   0.700000   

     water concentration wtf  dppc amount mass g  dspepeg2000 amount mass g  \
0                   0.999890             0.00006                   0.000028   
1                   0.963048             0.00006                   0.000028   
2                   0.926206             0.00006                   0.000028   
3                   0.889364             0.00006                   0.000028   
4                   0.852522             0.00006                   0.000028   
..                       ...                 ...                        ...   
495                 0.437258             0.00006                   0.000028   
496                 0.400416             0.00006                   0.000028   
497                 0.363574             0.00006                   0.000028   
498                 0.326732             0.00006                   0.000028   
499                 0.289890             0.00006                   0.000028   

     pfh amount mass g  ethanol amount mass g  water amount mass g  \
0                0.000               0.000000             0.799912   
1                0.000               0.029474             0.770438   
2                0.000               0.058947             0.740965   
3                0.000               0.088421             0.711491   
4                0.000               0.117895             0.682017   
..                 ...                    ...                  ...   
495              0.008               0.442105             0.349807   
496              0.008               0.471579             0.320333   
497              0.008               0.501053             0.290859   
498              0.008               0.530526             0.261386   
499              0.008               0.560000             0.231912   

     dppc amount volume mL  dspepeg2000 amount volume mL  \
0                      0.0                           0.0   
1                      0.0                           0.0   
2                      0.0                           0.0   
3                      0.0                           0.0   
4                      0.0                           0.0   
..                     ...                           ...   
495                    0.0                           0.0   
496                    0.0                           0.0   
497                    0.0                           0.0   
498                    0.0                           0.0   
499                    0.0                           0.0   

     pfh amount volume mL  ethanol amount volume mL  water amoun

In [ ]:
# use this for as a basis on how to build general stocks 
def stock_components(stock_name):
    """The stock name is required to be in the form 'solute n-solvent-stock' where the entry prior to the keyword stock are solvent
    and anything prior to that is assumed a solute. Will return a dictionary of the solvent and solute while pulling information from """
    stock_components = stock_name.split('-')
    stock_solutes = stock_components[:-2] # will always be a list
    stock_solvents = stock_components[-2]
    
    return stock_solutes, stock_solvents

def calculate_stock_prep_df(experiment_dict, volume_df, chem_database_path, buffer_pct = 40):
    
    # Isolate all stock volume entries in dataframe
    cols = volume_df.columns
    stock_cols = [col for col in cols if "stock" in col]
    stock_df = volume_df.copy()[stock_cols]
    
    # Compound volumes and add buffer
    stock_df.loc['Total Volume'] = stock_df.sum(numeric_only=True, axis=0)*(1+(buffer_pct/100))
    prep_df = pd.DataFrame(stock_df.loc['Total Volume']).T
    
    # Ensure all unit are same then convet to liters for calculations, latter is not 100% necessary
    check_unit_congruence(prep_df)
    prep_df = convert_to_liter(prep_df)
    
    # Add the concentration and respective units (may have this be arguments instead since only would be +1)
    prep_df.loc['Final Selected Stock Concentrations'] = experiment_dict['Final Selected Stock Concentrations']
    prep_df.loc['Stock Concentration Units'] = experiment_dict['Stock Concentration Units']
    
    chem_database_df = pd.read_excel(chem_database_path)
    
    prep_dicts = {}
    for stock in prep_df:
        total_volume = prep_df[stock]['Total Volume']
        stock_unit = prep_df[stock]['Stock Concentration Units']
        stock_conc = prep_df[stock]['Final Selected Stock Concentrations']
        solutes, solvent = stock_components(stock) # currently only one solvent and solute supported

        #All stocks will obvi have a solvent, but the solute is optional
        solvent_component_info = chem_database_df.loc[chem_database_df['Component Abbreviation'] == solvent]
        solvent_density = solvent_component_info['Density (g/L)'].iloc[0]

        if not solutes: # if no solutes present
            solute_mass = 0
            solute_volume = 0
            solvent_volume = total_volume
            solvent_mass = solvent_volume*solvent_density
            prep_dict = {'solute mass g': solute_mass,
               'solute volume L': solute_volume,
               'solvent mass g': solvent_mass,
               'solvent volume L': solvent_volume}

        if solutes: 
            solute = solutes[0]
            solute_component_info = chem_database_df.loc[chem_database_df['Component Abbreviation'] == solute] # add assertion to ensure component in database

            if stock_unit == 'molarity':
                solute_mw = solute_component_info['Molecular Weight (g/mol)'].iloc[0] # only call info if needed, if common between units then pull up one level
                solute_density = solute_component_info['Density (g/L)'].iloc[0]
                prep_dict = stock_molarity(total_volume, stock_conc, solute_mw, solute_density, solvent_density)

            if stock_unit == 'wtf':
                # since no density data available at the moment need to rough estimate, does not matter since the mass is scaled according to wtf, so long as more.
                solute_density = solute_component_info['Density (g/L)'].iloc[0]
                density_mix = bimixture_density_wtf(stock_conc, solute_density, solvent_density)
                total_mass = total_volume*density_mix
                prep_dict = stock_wtf(total_mass, stock_conc, 1-stock_conc, solute_density, solvent_density)
        prep_dicts[stock] = prep_dict
    stock_prep_df = pd.DataFrame.from_dict(prep_dicts) # add total volumes
    stock_complete_df = pd.concat([prep_df,stock_prep_df])
    return stock_prep_df

def isolate_common_column(df, common_string):
    cols = df.columns
    common_string_cols = [col for col in cols if common_string in col]
    final_df = df.copy()[common_string_cols]
    return final_df
